In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [2]:
# Definition of a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data_path, train_size):
        self.data = datasets.FashionMNIST(data_path, download=True, transform=transforms.ToTensor())
        self.train_size = train_size
        self.split_dataset()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        return image, label

    def split_dataset(self):
        num_train = int(self.train_size * len(self.data))
        self.data, _ = torch.utils.data.random_split(self.data, [num_train, len(self.data) - num_train])

# Definition of a custom collate function
def custom_collate(batch):
    images, labels = zip(*batch)
    images = torch.stack(images, dim=0)
    labels = torch.tensor(labels)
    return images, labels

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self, num_channels=32, kernel_size=3, pool_size=2):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, num_channels, kernel_size=kernel_size, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=pool_size, stride=pool_size)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(num_channels * 14 * 14, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.linear(x)
        return x

In [4]:
# Define the evaluate_model function
def evaluate_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = correct / total
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    return accuracy, precision, recall, f1

In [5]:
# Initialization of the list to store training losses for each hidden size, batch size, and training set size
batch_sizes = [4]
batch_size = 10
train_size = 0.9
train_losses_per_config = []
num_epochs = 1

num_channels = [10, 20, 100, 200]
kernel_sizes = [2, 3, (2, 2), (3, 3)]
pool_sizes = [2, (2, 2)]

In [6]:
# Definition of the train_model function
def train_model(model, train_loader, criterion, optimizer, device, num_epochs):
    train_losses = []
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0  # Initialize the epoch loss
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Add the batch loss to the epoch loss
            epoch_loss += loss.item()

        # Calculate the average loss for the epoch
        average_epoch_loss = epoch_loss / len(train_loader)

        # Append the average epoch loss to the list
        train_losses.append(average_epoch_loss)

        # Print the average epoch loss
        # print(f'Number of Channels: {num_channel}, Kernel Size: {kernel_size}, Batch Size: {batch_size}, Epoch [{epoch + 1}/{num_epochs}], Loss: {average_epoch_loss:.4f}')

    return train_losses

In [7]:
def do_model(num_channel, kernel_size,  pool_size):
    model = SimpleCNN(num_channels= num_channel, kernel_size=kernel_size, pool_size = pool_size)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    custom_dataset = CustomDataset('path', train_size)
    custom_data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)

    # Move the model to device (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Initialization of the list to store training losses
    # train_losses = []

    # Training the model
    train_losses_per_config.append(train_model(model, custom_data_loader, criterion, optimizer, device, num_epochs))

    # Evaluate the model and calculate metrics
    accuracy, precision, recall, f1 = evaluate_model(model, custom_data_loader, device)
    print(f'Number of Channels: {num_channel}, Kernel Size: {kernel_size}, Pool Size: {pool_size};\t Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

In [9]:
# Iterate over different configurations
for num_channel in num_channels:
    for kernel_size in kernel_sizes:
        for pool_size in pool_sizes:
            do_model(num_channel, kernel_size, pool_size)

Number of Channels: 10, Kernel Size: 2, Pool Size: 2;	 Accuracy: 0.8790, Precision: 0.8792, Recall: 0.8790, F1 Score: 0.8782
Number of Channels: 10, Kernel Size: 2, Pool Size: (2, 2);	 Accuracy: 0.8705, Precision: 0.8785, Recall: 0.8705, F1 Score: 0.8708
Number of Channels: 10, Kernel Size: 3, Pool Size: 2;	 Accuracy: 0.8801, Precision: 0.8835, Recall: 0.8801, F1 Score: 0.8774
Number of Channels: 10, Kernel Size: 3, Pool Size: (2, 2);	 Accuracy: 0.8893, Precision: 0.8888, Recall: 0.8893, F1 Score: 0.8887
Number of Channels: 10, Kernel Size: (2, 2), Pool Size: 2;	 Accuracy: 0.8773, Precision: 0.8774, Recall: 0.8773, F1 Score: 0.8743
Number of Channels: 10, Kernel Size: (2, 2), Pool Size: (2, 2);	 Accuracy: 0.8796, Precision: 0.8798, Recall: 0.8796, F1 Score: 0.8788
Number of Channels: 10, Kernel Size: (3, 3), Pool Size: 2;	 Accuracy: 0.8935, Precision: 0.8948, Recall: 0.8935, F1 Score: 0.8934
Number of Channels: 10, Kernel Size: (3, 3), Pool Size: (2, 2);	 Accuracy: 0.8914, Precision: 0

Przedstawione dane dotyczą wyników działania sieci neuronowej konwolucyjnej w różnych konfiguracjach, ze szczególnym uwzględnieniem różnych kombinacji liczby kanałów, rozmiaru kernela i rozmiaru puli.

Wpływ Rozmiaru Jądra:
Dla stałej liczby kanałów (10, 20, 100, 200), zwiększanie rozmiaru jądra z 2 do 3 powoduje stałą poprawę większości metryk wydajności.

Oddziaływanie Rozmiaru Puli:
Ogólnie rzecz biorąc, korzystanie z rozmiaru puli (2, 2) zazwyczaj skutkuje nieco lepszą wydajnością w porównaniu z rozmiarem puli równym 2. Ten trend jest zauważalny dla różnych rozmiarów jądra i liczby kanałów.

Wpływ Liczby Kanałów:
W miarę zwiększania liczby kanałów występuje mieszanka poprawy i pogorszenia metryk wydajności. Warto zauważyć, że większa liczba kanałów nie zawsze oznacza lepszą wydajność, a konieczne jest znalezienie optymalnego kompromisu.

Spójność Między Metrykami:
W różnych konfiguracjach występuje stosunkowo spójna wydajność we wszystkich metrykach, co wskazuje na zrównoważony model.


Najlepsza ogólna wydajność osiągana jest przy 100 kanałach, rozmiarze jądra 3 i rozmiarze puli 2, ponieważ systematycznie wykazuje wysokie wartości we wszystkich metrykach.